In [1]:
%pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 1.1 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/11.0 MB 1.3 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.0 MB 1.5 MB/s eta 0:00:07
   ----- ---------------------------------- 1.6/11.0 MB 1.5 MB/s eta 0:00:07
   ------ --------------------------------- 1.8/11.0 MB 1.4 MB/s eta 0:00:07
   ------- -------------------------------- 2.1/11.0 MB 1.4 MB/s eta 0:00:07
   -------- ------------------------------- 2.4/11.0 MB 1.4 MB/s eta 0:00:07
   -------- ------------------------------- 2.4/11.0 MB 1.4 MB/s eta 0:00:07
   ---------- ----------------------------- 2.9/11.0 MB 1.3 MB/s eta 0:00:07
   ---------- ----------------------------- 2.9/11.0 MB 1.3 MB/s eta 0:00:07
   ------------ --------------------------- 3.4/11.0 MB 1.3 MB/s eta 0:00:06
   ----------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd

df = pd.read_csv("r_orcamento.csv", encoding="latin-1")

,Unnamed: 0,Orçamento:,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,527576,Unnamed: 9,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Usuário:,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,ELISAMAP,Unnamed: 39
0,NaN,Situação:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Validade:,NaN,NaN,NaN,NaN,NaN,31/10/2025,NaN
1,NaN,Paciente:,NaN,NaN,NaN,NaN,NaN,NaN,107953,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Convênio:,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,...,NaN,31/10/2025,NaN,NaN,NaN,NaN,NaN,SEX,NaN,18 DIAS
3,Tipo Internação:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INDETERMINADO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Diagnóstico:,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,CURATIVOS ESPECIAIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,Procedimento,NaN,NaN,NaN,Unidade,NaN,NaN,Qtd,Dias,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,94115505,0000145304,"CURATIVO TEGADERM CHG 8,5X11,5CM",NaN,UN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sub-Total:,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
orcamento = df.columns[8]
data_orcamento = df.columns[20]
usuario = df.columns[38]
print("Orçamento: " + orcamento)
print("Data do Orçamento: " + data_orcamento)
print("Usuário: " + usuario)

# situacao = não preenchido
nr_carteira = df.iloc[0, 21] 
validade_carteira = df.iloc[0, 38]
nome_paciente = df.iloc[1, 11]
convenio = df.iloc[2, 10]
inicio =  df.iloc[2, 21]
termino =  df.iloc[2, 31]
complexidade =  df.iloc[3, 21]
total_periodo =  df.iloc[8, 12]
media_diaria =  df.iloc[8, 28]
print("Nro Carteira: " + str(nr_carteira))
print("Validade Carteira: " + validade_carteira)
print("Nome do paciente: " + nome_paciente)
print("Convenio: " + str(convenio))
print("Data de inicio: " + inicio)
print("Data de término: " + termino)
print("Complexidade: " + complexidade)
print("Total Período: " + total_periodo)
print("Média Período: " + media_diaria)


Orçamento: 527576
Data do Orçamento: Unnamed: 20
Usuário: ELISAMAP
Nro Carteira: 88888019062430021
Validade Carteira: 31/10/2025
Nome do paciente: VALERIA CRISTINA MARINHO DE OLIVEIRA QUEIROZ
Convenio: SULAMERICA
Data de inicio: 14/10/2025
Data de término: 31/10/2025
Complexidade: MEDIA COMPLEXIDADE
Total Período: 38.388,68
Média Período: 2132,70
